### Setup Conda Virtual Environment

IMPORTANT: Populate ```.env``` file in the home directory of your repository. Use the [.sample.env](.sample.env) as a reference guide.

In [1]:
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv(), override=True)
print(os.getenv('AZURE_OPENAI_ENDPOINT'))

https://lllmops-aoai-sweden-c.openai.azure.com/


From the home directory run the folloing bash commands

```bash 
conda env create --name llmops --file=./conda.yaml
```  

```bash
conda activate llmops
```

### Get a Handle to the AML Workspace

In [2]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import AmlCompute


# authenticate
credential = DefaultAzureCredential(tenantid=os.environ.get('TENANT_ID'))

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id = os.environ.get('SUBSCRIPTION_ID'),
    resource_group_name = os.environ.get('RESOURCE_GROUP_NAME'),
    workspace_name = os.environ.get('WORKSPACE_NAME'),
)

### Create Local PromptFlow Connection to AOAI

In [ ]:
# Note: Optional (only needed for local development/testing)

import subprocess
import os

api_key = os.getenv("AZURE_OPENAI_KEY")
api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
name = "aoai-connect"

command = f"pf connection create --file src/promptflows/connections/connect_aoai.yaml --set api_key={api_key} api_base={api_base} --name {name}"
result = subprocess.run(command,
                        shell=True,
                        check=True,
                        text=True,
                        capture_output=True)

print(result.stdout)

In [ ]:
!pf connection list

### Create a Remote Connection to AOAI

Navigate to the Azure Machine Learning Workspace --> Promptflow --> connections --> Create
Then follow the steps to create a connection

**IMPORTANT:** Name your connection 'aoai-connect'

In [7]:
## NOT YET SUPPORTED ##
'''
from promptflow.azure import PFClient
from promptflow.entities import AzureOpenAIConnection

pf = PFClient(credential=DefaultAzureCredential(),
              subscription_id=os.environ.get('SUBSCRIPTION_ID'),
              resource_group_name=os.environ.get('RESOURCE_GROUP_NAME'),
               workspace_name=os.environ.get('WORKSPACE_NAME')
             )

aoai_connection = AzureOpenAIConnection(api_base=os.environ.get('AZURE_OPENAI_ENDPOINT'), 
                                        api_key=os.environ.get('AZURE_OPENAI_API_KEY'), 
                                        name="aoai-connect", 
                                        api_type = "azure", 
                                        api_version="2024-02-01"
                                        )

result = pf.connections.create_or_update(aoai_connection)
print(result)
'''

'\nfrom promptflow.azure import PFClient\nfrom promptflow.entities import AzureOpenAIConnection\n\npf = PFClient(credential=DefaultAzureCredential(),\n              subscription_id=os.environ.get(\'SUBSCRIPTION_ID\'),\n              resource_group_name=os.environ.get(\'RESOURCE_GROUP_NAME\'),\n               workspace_name=os.environ.get(\'WORKSPACE_NAME\')\n             )\n\naoai_connection = AzureOpenAIConnection(api_base=os.environ.get(\'AZURE_OPENAI_ENDPOINT\'), \n                                        api_key=os.environ.get(\'AZURE_OPENAI_API_KEY\'), \n                                        name="aoai-connect", \n                                        api_type = "azure", \n                                        api_version="2024-02-01"\n                                        )\n\nresult = pf.connections.create_or_update(aoai_connection)\nprint(result)\n'

### Create a Remote Compute Target

In [ ]:
# Create Compute Target

# Name assigned to the compute cluster
cpu_compute_target = "remote-cpu-cluster"

try:
    # let's see if the compute target already exists
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(f"You already have a cluster named {cpu_compute_target}, we'll reuse it as is.")

except Exception:
    print("Creating a new cpu compute target...")
    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        type="amlcompute",
        size="STANDARD_DS3_V2",
        min_instances=0,
        max_instances=1,
        idle_time_before_scale_down=300, # 5 minutes
        tier="Dedicated",
    )
    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster)

print(cpu_cluster)